In [1]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import optuna
from optuna.samplers import TPESampler
#import lightgbm as lgbm
#from xgboost import XGBRegressor
import xgboost as xgb
#from catboost import CatBoostRegressor
#from lightgbm.sklearn import LGBMRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
pd.set_option('display.max_columns', 500)
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
from sklearn import metrics

import sys, os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import math
import pickle
import mplcyberpunk

In [2]:
from itertools import islice, cycle

plt.style.use("cyberpunk")

def add_secondary_plot(df, column, target_column, ax, n_bins, color=3, show_yticks=False, marker="."):
    secondary_ax = ax.twinx()
    bins = pd.cut(df[column], bins=n_bins)
    bins = pd.IntervalIndex(bins)
    bins = (bins.left + bins.right) / 2
    target = df.groupby(bins)[target_column].mean()
    target.plot(
        ax=secondary_ax, linestyle='',
        marker=marker, color=color, label=f"Mean '{target_column}'"
    )
    secondary_ax.grid(visible=False)
    
    if not show_yticks:
        secondary_ax.get_yaxis().set_ticks([])
        
    return secondary_ax

def render_feature_distros(train_df, test_df, features=[], labels=[], n_bins=50, n_cols=4, pad=2, h_pad=4, w_pad=None):
    histplot_hyperparams = {
        'kde':True,
        'alpha':0.4,
        'stat':'percent',
        'bins':n_bins
    }
    markers = ['.', '+', 'x', '1', '2']
    
    n_rows = math.ceil(len(features) / n_cols)
    cell_with_dim = 4
    cell_height_dim = 3
    
    fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * cell_with_dim, n_rows * cell_height_dim))
    plt.tight_layout(pad=pad, h_pad=h_pad, w_pad=w_pad, rect=None)
    
    # delete exess subplots
    for a in ax[n_rows - 1, int(((n_rows - (len(features) / n_cols)) * n_cols*-1)):]:
        a.axis('off')
        
    leg_handles = []
    leg_labels = []
    
    axs = []

    for i, feature in enumerate(features):
        row = math.ceil(i / n_cols) - 1
        col = (i % n_cols)
        
        color_cycle = islice(mplcyberpunk.cyberpunk_stylesheets['cyberpunk']['axes.prop_cycle'], 0, None)
        
        sns.histplot(train_df[feature], label='Train X', ax=ax[row, col], color=next(color_cycle)['color'], **histplot_hyperparams)
        sns.histplot(test_df[feature], label='Test X', ax=ax[row, col],color=next(color_cycle)['color'], **histplot_hyperparams)
        ax[row, col].set_title(f'{feature} Distribution')
        mplcyberpunk.make_lines_glow(ax[row, col])
        axs.append(ax[row, col].get_legend_handles_labels())

        for j, label in enumerate(labels):
            sub_ax = add_secondary_plot(train_df, feature, label, ax[row, col], n_bins, color=next(color_cycle)['color'], marker=markers[j])
            axs.append(sub_ax.get_legend_handles_labels())
        
    for axis in axs:
        if axis[1][0] not in leg_labels:
            leg_labels.extend(axis[1])
            leg_handles.extend(axis[0])
        
    fig.legend(leg_handles, leg_labels, loc='upper center', bbox_to_anchor=(0.5, 1.04), fontsize=14, ncol=len(features) + 2)

In [3]:
train_time = 1 * 60 * 60
objective = 'xgbregressor'

In [4]:
sys.path.append(os.path.realpath('..'))

In [5]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
sample_sub = pd.read_csv(r"..\data\sample_submission.csv", index_col=0)

In [6]:
total = train_df.isnull().sum().sort_values(ascending=False)
percent = (train_df.isnull().sum()/train_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

,Total,Percent
Age,0,0.0
StockOptionLevel,0,0.0
Over18,0,0.0
OverTime,0,0.0
PercentSalaryHike,0,0.0
PerformanceRating,0,0.0
RelationshipSatisfaction,0,0.0
StandardHours,0,0.0
TotalWorkingYears,0,0.0
BusinessTravel,0,0.0


In [7]:
del train_df['Over18']
del train_df['EmployeeCount']
del train_df['StandardHours']

In [8]:
columns_to_vectorize = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
for vector_target in columns_to_vectorize:
    print(vector_target)
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

BusinessTravel
Department
EducationField
Gender
JobRole
MaritalStatus
OverTime


In [9]:
features = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction','HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
        'BusinessTravel_v', 'Department_v', 'EducationField_v','Gender_v', 'JobRole_v', 'MaritalStatus_v', 'OverTime_v']
target = ['Attrition']

In [10]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [11]:
RANDOM_STATE = 12 
FOLDS = 5
param_grid_history = {}


def objective_v3(trial):
    oof = np.zeros(len(X))
    scores = []
    param_grid = {
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'n_estimators': trial.suggest_int('n_estimators', 2, 1000, 1), 
        #'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.9),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 0.9),
        'eta': trial.suggest_float('eta', 0.001, 0.9), 
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 5), 
    } 
        
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        print(10*"=", f"Fold={fold+1}", 10*"=")
        X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
        y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 
        
        reg = xgb.XGBModel(
            # These parameters should help with trial speed.
            objective='binary:logistic',
            tree_method='gpu_hist',
            booster='gbtree',
            predictor='gpu_predictor',
            n_jobs=4,
            eval_metric='auc',
            early_stopping_rounds=100,
            **param_grid
        )

        reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
            
        oof[valid_idx] = reg.predict(X_valid)

    roc_auc = roc_auc_score(Y, oof)
    
    param_grid_history[roc_auc] = param_grid

    return roc_auc

In [12]:
param_grid_history = {}

def skfold_cv(model, X, y, metric, n_folds=69):
    skfold = StratifiedKFold(n_splits=n_folds, random_state=0, shuffle=True)
    scores = []
    for train_index, test_index in skfold.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0)
        predictions = model.predict_proba(X_test.values)
        
        score = metric(y_test, predictions[:, 1])
        scores.append(score)
        print(f'ROC AUC: {score:.3f}')
    return scores

def objective_v4(trial):
    param_grid = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.05),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000),
        "n_estimators": trial.suggest_int('num_boost_round', 500, 2000),
        'tree_method': 'gpu_hist',
        'n_jobs': -1,
        'early_stopping_rounds': 100
    }
    model = xgb.XGBClassifier(**param_grid)
    scores = skfold_cv(model, train_df[features], train_df[target], roc_auc_score, 5)
    score = np.sum(scores)/5
    param_grid_history[score] = param_grid
    return score

In [13]:
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name=objective) #recommeded between 100-1000 itterations
study.optimize(objective_v4, n_trials=1000)

[I 2023-01-21 21:31:48,211] A new study created in memory with name: xgbregressor


ROC AUC: 0.831
ROC AUC: 0.852
ROC AUC: 0.833
ROC AUC: 0.826


[I 2023-01-21 21:32:01,740] Trial 0 finished with value: 0.8372488547869903 and parameters: {'max_depth': 8, 'learning_rate': 0.010674578185900327, 'min_child_weight': 4, 'gamma': 0.5657752183677626, 'subsample': 0.47370149164399633, 'colsample_bytree': 0.21325271660376843, 'reg_alpha': 0.5145198389793625, 'reg_lambda': 0.642438763705907, 'random_state': 177, 'num_boost_round': 851}. Best is trial 0 with value: 0.8372488547869903.


ROC AUC: 0.845
ROC AUC: 0.826
ROC AUC: 0.863
ROC AUC: 0.828
ROC AUC: 0.807


[I 2023-01-21 21:32:11,252] Trial 1 finished with value: 0.8347364292258359 and parameters: {'max_depth': 5, 'learning_rate': 0.01840042666421507, 'min_child_weight': 9, 'gamma': 0.944064773496654, 'subsample': 0.9646940514680609, 'colsample_bytree': 0.07798411723742764, 'reg_alpha': 0.9428731837118646, 'reg_lambda': 0.0351357318417761, 'random_state': 186, 'num_boost_round': 780}. Best is trial 0 with value: 0.8372488547869903.


...

[I 2023-01-21 21:45:44,692] Trial 88 finished with value: 0.8454434264773247 and parameters: {'max_depth': 3, 'learning_rate': 0.026757658287762012, 'min_child_weight': 2, 'gamma': 0.7478138933297798, 'subsample': 0.6499656568001234, 'colsample_bytree': 0.16501750948642874, 'reg_alpha': 0.8518086327975358, 'reg_lambda': 0.07649876544851675, 'random_state': 553, 'num_boost_round': 908}. Best is trial 87 with value: 0.8480236486486487.


ROC AUC: 0.852
ROC AUC: 0.835
ROC AUC: 0.874
ROC AUC: 0.850
ROC AUC: 0.818


[I 2023-01-21 21:45:55,340] Trial 89 finished with value: 0.8464852840128264 and parameters: {'max_depth': 3, 'learning_rate': 0.029324195247425796, 'min_child_weight': 2, 'gamma': 0.7842697931669004, 'subsample': 0.6344419846605337, 'colsample_bytree': 0.03584641859016507, 'reg_alpha': 0.8581397178803952, 'reg_lambda': 0.0712697320686891, 'random_state': 551, 'num_boost_round': 928}. Best is trial 87 with value: 0.8480236486486487.


ROC AUC: 0.856


[W 2023-01-21 21:45:56,497] Trial 90 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "D:\source\repos\venv\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_14704\3181594328.py", line 35, in objective_v4
    scores = skfold_cv(model, train_df[features], train_df[target], roc_auc_score, 5)
  File "C:\Users\ryans\AppData\Local\Temp\ipykernel_14704\3181594328.py", line 10, in skfold_cv
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0)
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\sklearn.py", line 1490, in fit
    self._Booster = train(
  File "D:\source\repos\venv\Python310\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File

In [14]:
parm_grid_bytes = pickle.dumps(param_grid_history)
with open(f"rendered_data/{objective}_bytes.hex", "wb") as binary_file:
    binary_file.write(parm_grid_bytes)

In [15]:
def train(param_grid):
    reg = xgb.XGBModel(
        # These parameters should help with trial speed.
        objective='binary:logistic',
        booster='gbtree',
        predictor='gpu_predictor',
        eval_metric='auc',
        **param_grid
    )
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.20)
    
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
        
    return reg

percent = math.ceil(len(param_grid_history.keys()) * .1)
top = sorted(list(param_grid_history.keys()))[-1:]

train_preds = []
test_preds = []
for key in tqdm(top):
    model = train(param_grid_history[key])
    train_preds.append(model.predict(train_df[features]))
    test_preds.append(model.predict(test_df[features]))

train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.93it/s]


In [16]:
train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
train_submission.to_csv(fr'rendered_data/{objective}_train_submission.csv', index=False)

test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
test_submission.to_csv(fr'rendered_data/{objective}_test_submission.csv', index=False)